In [8]:
json_dict = """{
  "nodes": {
    "__START__": {
      "schema_info": "None",
      "input_schema": "None",
      "output_schema": "None",
      "description": "The starting point of the graph.",
      "function_name": "start_node"
    },
    "supervisor": {
      "schema_info": "Router: TypedDict with next as a string literal of options.",
      "input_schema": "State",
      "output_schema": "State",
      "description": "This node manages the conversation between workers and decides which worker to route to next.",
      "function_name": "supervisor_node"
    },
    "food_logger": {
      "schema_info": "State: TypedDict with messages as a list of messages.",
      "input_schema": "State",
      "output_schema": "State",
      "description": "This node logs food intake and calculates calories.",
      "function_name": "food_logger_node"
    },
    "workout_tracker": {
      "schema_info": "State: TypedDict with messages as a list of messages.",
      "input_schema": "State",
      "output_schema": "State",
      "description": "This node logs workout details and calculates calories burnt.",
      "function_name": "workout_tracker_node"
    },
    "__END__": {
      "schema_info": "None",
      "input_schema": "None",
      "output_schema": "None",
      "description": "The endpoint of the graph, indicating completion.",
      "function_name": "end_node"
    }
  },
  "edges": {
    { 
      "source": "__START__", 
      "target": "supervisor", 
      "routing_conditions": "Start the process by routing to the supervisor node.", 
      "conditional": False 
    },
    { 
      "source": "supervisor", 
      "target": "food_logger", 
      "routing_conditions": "If the supervisor decides to route to food_logger based on user input.", 
      "conditional": True 
    },
    { 
      "source": "supervisor", 
      "target": "workout_tracker", 
      "routing_conditions": "If the supervisor decides to route to workout_tracker based on user input.", 
      "conditional": True 
    },
    { 
      "source": "supervisor", 
      "target": "__END__", 
      "routing_conditions": "If the supervisor decides to finish the process.", 
      "conditional": True
    },
    { 
      "source": "food_logger", 
      "target": "supervisor", 
      "routing_conditions": "After logging food intake, return control to the supervisor.", 
      "conditional": False 
    },
    { 
      "source": "workout_tracker", 
      "target": "supervisor", 
      "routing_conditions": "After logging workout details, return control to the supervisor.", 
      "conditional": False 
    }
  }
}
"""

In [23]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import SystemMessage, HumanMessage

SYSTEM_PROMPT = ChatPromptTemplate.from_template("""
You are given a json representing a AI workflow graph,
                                                 
you are also provided certain use cases given by the user that the workflow would need to take care of.

provide a step-by-step example of how the state changes or how the routing decisions are made for these inputs
Evaluate if any use cases are not handled by the graph, regenerate the graph in that case                                                                                                
<Graph>
{json_dict}                                             
</Graph>

<UseCases>
1. I ate 10 cookies today
2. I ran 30 miles today
3. Is ice cream good for my health, based on my monthly goals?
</UseCases>
""")

In [24]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0)

In [25]:
result = llm.invoke([HumanMessage(content=SYSTEM_PROMPT.format(json_dict=json_dict))])

In [26]:
result.pretty_print()

================================== Ai Message ==================================

Let's analyze the given AI workflow graph and the provided use cases step-by-step, showing how the state changes and routing decisions are made. Then, we will evaluate if all use cases are handled by the graph and regenerate the graph if needed.

---

## Given Graph Summary

- **Nodes:**
  - `__START__`: Entry point.
  - `supervisor`: Routes to workers based on input.
  - `food_logger`: Logs food intake and calculates calories.
  - `workout_tracker`: Logs workout details and calculates calories burnt.
  - `__END__`: Endpoint.

- **Edges:**
  - `__START__` → `supervisor` (unconditional)
  - `supervisor` → `food_logger` (conditional)
  - `supervisor` → `workout_tracker` (conditional)
  - `supervisor` → `__END__` (conditional)
  - `food_logger` → `supervisor` (unconditional)
  - `workout_tracker` → `supervisor` (unconditional)

---

## Use Cases

1. **"I ate 10 cookies today"**  
   User reports food intake.

# insights

1. use cases are propely run against the graph
2. it was identified that the 3rd use case will not work
3. the graph was correctly fixed